# Demo for SFLKit

## What is SFLKit

In [1]:
!pip install .

Processing /Users/marius/Desktop/work/projects/statistical-debugger
  Preparing metadata (setup.py) ... done
  Created wheel for sflkit: filename=sflkit-0.0.1-py3-none-any.whl size=32964 sha256=2b1aefebb83a077aac6dda53c53c9914537f95374750eef5e63337948e7431e6
  Stored in directory: /Users/marius/Library/Caches/pip/wheels/57/cc/c6/b7063d147bc0b12a7f6ee2dd2ffd25d696d6fbcc41b76dfb97
Successfully built sflkit
  Attempting uninstall: sflkit
    Found existing installation: sflkit 0.0.1
    Uninstalling sflkit-0.0.1:
      Successfully uninstalled sflkit-0.0.1


## A faulty Program

In [29]:
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y  # bug
    else:
        if x > y:
            m = y
        elif x > z:
            m = x
    return m

In [30]:
import enum

In [31]:
class TestResult(enum.Enum):
    PASS = 'PASS'
    FAIL = 'FAIL'

In [32]:
def test_middle(x, y, z, expected):
    try:
        if middle(x, y, z) == expected:
            return TestResult.PASS
        else:
            return TestResult.FAIL
    except BaseException:
        return TestResult.FAIL

In [33]:
test_middle(3, 2, 1, expected=2)

<TestResult.PASS: 'PASS'>

In [34]:
test_middle(3, 1, 2, expected=2)

<TestResult.PASS: 'PASS'>

In [35]:
test_middle(2, 1, 3, expected=2)

<TestResult.FAIL: 'FAIL'>

## Instrument the Program

In [36]:
import inspect

In [37]:
source = inspect.getsource(middle)
print(source)

def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y  # bug
    else:
        if x > y:
            m = y
        elif x > z:
            m = x
    return m



In [38]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'

In [39]:
with open(middle_py, 'w') as fp:
    fp.write(source)

In [40]:
def test_middle_import(x, y, z, expected):
    from middle import middle
    try:
        if middle(x, y, z) == expected:
            return TestResult.PASS
        else:
            return TestResult.FAIL
    except BaseException:
        return TestResult.FAIL

In [41]:
test_middle_import(3, 2, 1, expected=2)

<TestResult.PASS: 'PASS'>

In [42]:
test_middle_import(3, 1, 2, expected=2)

<TestResult.PASS: 'PASS'>

In [43]:
test_middle_import(2, 1, 3, expected=2)

<TestResult.FAIL: 'FAIL'>

In [44]:
from sflkit.language import Language, meta
from sflkit.instrumentation.dir_instrumentation import DirInstrumentation
from sflkit.events import EventType

In [45]:
python = Language.PYTHON
python.setup()

In [46]:
line_visitor = python.meta_visitors[EventType.LINE]

In [47]:
def instrument(visitor):
    instrumentation = DirInstrumentation(python.visitor(visitor(python, meta.IDGenerator(), meta.TmpGenerator())))
    instrumentation.instrument(src=middle_py, dst=tmp_py, suffixes=python.suffixes)
    with open(tmp_py, 'r') as fp:
        print(fp.read())

In [48]:
instrument(line_visitor)

import sflkit.instrumentation.lib


def middle(x, y, z):
    sflkit.instrumentation.lib.add_line_event('middle.py', 2, 0)
    m = z
    sflkit.instrumentation.lib.add_line_event('middle.py', 3, 1)
    if y < z:
        sflkit.instrumentation.lib.add_line_event('middle.py', 4, 2)
        if x < y:
            sflkit.instrumentation.lib.add_line_event('middle.py', 5, 3)
            m = y
        else:
            sflkit.instrumentation.lib.add_line_event('middle.py', 6, 4)
            if x < z:
                sflkit.instrumentation.lib.add_line_event('middle.py', 7, 5)
                m = y
    else:
        sflkit.instrumentation.lib.add_line_event('middle.py', 9, 6)
        if x > y:
            sflkit.instrumentation.lib.add_line_event('middle.py', 10, 7)
            m = y
        else:
            sflkit.instrumentation.lib.add_line_event('middle.py', 11, 8)
            if x > z:
                sflkit.instrumentation.lib.add_line_event('middle.py', 12, 9)
                m = x
    

In [49]:
def test_tmp(x, y, z, expected): 
    import tmp
    tmp.sflkit.instrumentation.lib.reset()
    try:
        if tmp.middle(x, y, z) == expected:
            return TestResult.PASS
        else:
            return TestResult.FAIL
    except BaseException:
        return TestResult.FAIL
    finally:
        del tmp

In [50]:
import os, shutil
from sflkit.model.event_file import EventFile
event_files = 'event-files'

In [51]:
def run_tests():
    if os.path.exists(event_files):
        shutil.rmtree(event_files)
    os.mkdir(event_files)
    test_tmp(3, 2, 1, expected=2)
    shutil.move('EVENTS_PATH', os.path.join(event_files, '0'))
    test_tmp(3, 1, 2, expected=2)
    shutil.move('EVENTS_PATH', os.path.join(event_files, '1'))
    test_tmp(2, 1, 3, expected=2)
    shutil.move('EVENTS_PATH', os.path.join(event_files, '2'))
    passing = [EventFile(os.path.join(event_files, '0'), 0, failing=False), EventFile(os.path.join(event_files, '1'), 1, failing=False)]
    failing = [EventFile(os.path.join(event_files, '2'), 2, failing=True)]
    return passing, failing

In [52]:
p, f = run_tests()

In [53]:
p

[event-files/0:0:PASS, event-files/1:1:PASS]

In [54]:
f

[event-files/2:2:FAIL]